# Emma Grimm, Assignment 4

extracting shape information from a font to render text in unexpected ways

In [586]:
from flat import document, rgb, rgba, font, strike, shape, command
from flat.command import moveto, quadto, curveto, lineto, closepath
from bezmerizing import Polyline, Path
import numpy as np
from copy import copy
import unicodedata
import opensimplex
from numpy.random import uniform, normal, choice
import json
from itertools import chain

In [587]:
from IPython.display import SVG, display
def show(page):
    display(SVG(page.svg()))

I used some of the functions from the *Manipulating Font Data* notebook, and modified others to fit my purposes.

In [525]:
def glyphcommands(f, ch):
    return Path([copy(cmd) for cmd in f.glyph(f.charmap[ord(ch)])])
def advancefor(f, ch):
    return f.advances[f.charmap[ord(ch)]]

In [526]:
from itertools import chain
def combine_path(f, s):
    text_paths = []
    cx = 0
    for ch in s:
        glyph_path = glyphcommands(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

After going through the *Quick Hershey Fonts* notebook, I was interested in seeing if anything cool could be made using noise with non-Hershey fonts. I modified the nudge function in order to try it out.

In [527]:
def nudgeBase(cmd, x, y):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += x
        c.y += y
    if type(cmd) in (quadto, curveto):
        c.x1 += x
        c.y1 += y
    if type(cmd) == curveto:
        c.x2 += x
        c.y2 += y
    return c

In [528]:
def nudge(cmd, scale, noiseLevel):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += noisy.noise3d(c.x*noiseLevel, c.y*noiseLevel, 2) * scale
        c.y += noisy.noise3d(c.x*noiseLevel, c.y*noiseLevel, 102) * scale
    if type(cmd) in (quadto, curveto):
        c.x1 += noisy.noise3d(c.x1*noiseLevel, c.y1*noiseLevel, 2) * scale
        c.y1 += noisy.noise3d(c.x1*noiseLevel, c.y1*noiseLevel, 102) * scale
        c.y *= 1.5
        c.y1 *= 1.5
    if type(cmd) == curveto:
        c.x2 += noisy.noise3d(c.x2*noiseLevel, c.y2*noiseLevel, 2) * scale
        c.y2 += noisy.noise3d(c.x2*noiseLevel, c.y2*noiseLevel, 102) * scale
    if type(cmd) == lineto:
        c.x *= 1.5
    return c

In [529]:
f = font.open("./Lobster-Regular.ttf")

noisy = opensimplex.OpenSimplex()

In [530]:
d_path_orig = glyphcommands(f, "d")

In [531]:
d_path_mod = Path([nudge(cmd, scale = 50, noiseLevel = 0.01) for cmd in b_path_orig])

In [532]:
page = document(288, 144, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(d_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(d_path_mod.scale(144/f.density).translate(144, 144)))
show(page)

I actually sort of like how this looks! However, because of the way Lobster looks as a font originally, there are a bunch of straight lines that are preserved and kind of interrupt the blobiness. So I wanted to try transforming the whole shape into quad beziers, rather than a mixture of lines and curves.

In [533]:
def hersh(f, c):
    glyph_path = glyphcommands(f, c)
    cmds = []
    for cmd in glyph_path:
        if type(cmd) == moveto:
            cmds.append(moveto(cmd.x, cmd.y))
        if type(cmd) == lineto:
            cmds.append(lineto(cmd.x, cmd.y))
        if type(cmd) == quadto:
            cmds.append(lineto(cmd.x, cmd.y))
        if type(cmd) == closepath:
            cmds.append(closepath)
    newPath = Path(cmds)
    return newPath

In [534]:
d = document(288, 144, 'pt')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).nofill()
factor = 144 / f.density

letter = hersh(f, "d")

page.place(pen.path(letter.scale(factor).translate(100, 150)))
show(page)

First, I tried going the other way to make sure I understood how all the functions worked. The **hersh** function above converts the font into a kind of hershey font, that is, it's made up purely of straight lines. After this I felt ready to tackle the curves!

In [535]:
from math import sqrt
def sails(f, c):
    glyph_path = glyphcommands(f, c)
    cmds = []
    for cmd in glyph_path:
        if type(cmd) == moveto:
            cmds.append(moveto(cmd.x, cmd.y))
        if type(cmd) == lineto:
            cmds.append(quadto(sqrt(sqrt(cmd.x*cmd.x)), cmd.y+normal(0,2), cmd.x, cmd.y))
        if type(cmd) == quadto:
            cmds.append(quadto(cmd.x1-normal(150,50), cmd.y1+normal(0,2), cmd.x, cmd.y))
        if type(cmd) == closepath:
            cmds.append(closepath)
    newPath = Path(cmds)
    return newPath

In [536]:
d = document(288, 144, 'pt')
page = d.addpage()
pen = shape().stroke(rgba(0, 0, 0, 255)).nofill()
factor = 144 / f.density

letter = sails(f, "d")

page.place(pen.path(letter.scale(factor).translate(100, 150)))
show(page)

The **sails** function converts every straight line into a quadratic bezier curve that curves towards the left side of the page. I think this makes the letter look like a series of sails blowing in the wind. I modified the curves a bit to have a similar shape, so that there is the suggestion of a wind from the east. I also added a bit of deviation to the y values of the control points (and the x values for the lines that were already curves) to make it look a bit less flat and give the shape more life. I then modified the **combine_path** function to combine **sails**-modified characters instead of regular characters.

In [537]:
def combine_path(f, s):
    text_paths = []
    cx = 100
    for ch in s:
        glyph_path = sails(f, ch).translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch)
    combined = Path(list(chain(*text_paths)))
    return combined

In [538]:
page = document(600, 400, 'pt').addpage()
pen = shape().stroke(rgba(161, 187, 209, 128)).fill(rgba(161, 187, 209, 255))
factor = 72 / f.density

page.place(pen.path(combine_path(f, "And the wayward wind").scale(factor).translate(0, 90)))
page.place(pen.path(combine_path(f, "is a restless wind").scale(factor).translate(0, 180)))
page.place(pen.path(combine_path(f, "A westward wind").scale(factor).translate(0, 270)))
page.place(pen.path(combine_path(f, "that likes to wander").scale(factor).translate(0, 360)))

show(page)

Finally, I used the modified font to display some slightly altered lyrics from Gogi Grant's *The Wayward Wind*. I thought it matched the feeling that I got from the font after messing with it. Maybe I'm still stuck on concrete poetry?

In [539]:
f = font.open("./Pacifico-Regular.ttf")

In [540]:
def nudge(cmd, scale, noiseLevel):
    c = copy(cmd)
    if type(cmd) in (moveto, lineto, quadto, curveto):
        c.x += noisy.noise3d(c.x*noiseLevel, c.y*noiseLevel, 2) * scale
        c.y += noisy.noise3d(c.x*noiseLevel, c.y*noiseLevel, 102) * scale
        c.y *= 1.5
    if type(cmd) in (quadto, curveto):
        c.x1 += noisy.noise3d(c.x1*noiseLevel, c.y1*noiseLevel, 2) * scale
        c.y1 += noisy.noise3d(c.x1*noiseLevel, c.y1*noiseLevel, 102) * scale
        c.y1 *= 1.5
    if type(cmd) == curveto:
        c.x2 += noisy.noise3d(c.x2*noiseLevel, c.y2*noiseLevel, 2) * scale
        c.y2 += noisy.noise3d(c.x2*noiseLevel, c.y2*noiseLevel, 102) * scale
    return c

In [541]:
d_path_orig = glyphcommands(f, "d")

In [542]:
d_path_mod = Path([nudge(cmd, scale = 50, noiseLevel = 0.01) for cmd in b_path_orig])

In [543]:
page = document(288, 200, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))
page.place(pen.path(b_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(b_path_mod.scale(144/f.density).translate(144, 250)))
show(page)

Loaded in a new font that's all curves so I could play with the blobbiness.

In [563]:
def stretch_char(f, inPath, height=-0.2):
    path_orig = inPath
    path_cmds = []
    mid_y = f.ascender * height
    for cmd in inPath:
        if hasattr(cmd, 'y') and cmd.y > mid_y:
            path_cmds.append(nudgeBase(cmd, x=normal(0,30), y=sqrt(cmd.y-mid_y)*30 + normal(200,100)))
        else:
            path_cmds.append(cmd)
    path_mod = Path(path_cmds)
    return path_mod

In [564]:
page = document(288, 300, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40)).fill(rgb(240, 240, 240))

b_path_stretch = stretch_char(f, b_path_mod)

page.place(pen.path(b_path_orig.scale(144/f.density).translate(0, 144)))
page.place(pen.path(b_path_stretch.scale(144/f.density).translate(144, 220)))

show(page)

I wanted a kind of melting effect so I slightly modified the **strech_char** function so that the character gets more and more stretched out the further it goes below the middle-ish point.

In [565]:
def combine_path(f, s):
    text_paths = []
    cx = 100
    for ch in s:
        mod_path = Path([nudge(cmd, scale = 50, noiseLevel = 0.01) for cmd in glyphcommands(f, ch)])
        stretched_path = stretch_char(f, mod_path)
        glyph_path = stretched_path.translate(cx, 0)
        text_paths.append(glyph_path)
        cx += advancefor(f, ch) + 50
    combined = Path(list(chain(*text_paths)))
    return combined

In [566]:
page = document(600, 400, 'pt').addpage()
alpha = 255
pen = shape().stroke(rgba(0, 0, 0, alpha)).fill(rgba(0, 0, 0, alpha))
factor = 72 / f.density

for i in range(3):
    pen = pen.stroke(rgba(0, 0, 0, alpha)).fill(rgba(0, 0, 0, alpha))
    page.place(pen.path(combine_path(f, "mind is melting").scale(factor).translate(0, 200)))
    alpha -= 50

show(page)

Finally, I modified the combine_path function again to make the melting phrase with some variation in character transformation using normal(). I like both the version above and below for the final composition, I think the idea of melting letters is decently communicated through font manipulation.

In [568]:
page = document(600, 400, 'pt').addpage()
factor = 72 / f.density

for i in range(3):
    alpha = 255 - i*100
    pen = shape().stroke(rgba(0, 0, 0, alpha)).fill(rgba(0, 0, 0, alpha))
    page.place(pen.path(combine_path(f, "mind is melting").scale(factor).translate(i*20, 200+(i*20))))

show(page)

In [618]:
def scribble(f, c):
    glyph_path = glyphcommands(f, c)
    pts = []
    start = []
    last = []
    for cmd in glyph_path:
        if type(cmd) == moveto:
            start = [cmd.x, cmd.y]
            last = [cmd.x, cmd.y]
            pts.append(start)
            
        elif type(cmd) == closepath:
            pts.append(start)
            
        elif hasattr(cmd, "x"):
            for i in range(5):
                pts.append([last[0] + ((cmd.x-last[0])/10)*i + normal(0,10), last[1] + ((cmd.y-last[1])/10)*i + normal(0,10)])
            pts.append([cmd.x, cmd.y])  
            last = [cmd.x, cmd.y]
            
    newPline = Polyline(pts)
    return newPline

I couldn't figure out why I was still getting the closepath has no x attribute error after putting in elif type(cmd) == closepath, but I ended up having to add hasattr(cmd, "x") just to get rid of it.

In [619]:
page = document(288, 200, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40))
factor = 144 / f.density

letter = scribble(f, "h").scale(factor).translate(100,200)
letter2 = scribble(f, "h").smooth_path().scale(factor).translate(200,200)

lines = pen.polyline(letter)
curve = pen.path(letter2)

page.place(lines)
page.place(curve)

show(page)

In [622]:
def scribble(f, c):
    glyph_path = glyphcommands(f, c)
    pts = []
    start = []
    last = []
    for cmd in glyph_path:
        if type(cmd) == moveto:
            start = [cmd.x, cmd.y]
            last = [cmd.x, cmd.y]
            pts.append(start)

        elif type(cmd) == closepath:
            pts.append(start)
            
        elif hasattr(cmd, "x"):
            for i in range(10):
                pts.append([last[0] + ((cmd.x-last[0])/10)*i + normal(0,30), last[1] + ((cmd.y-last[1])/50)*i + normal(0,30)])
            pts.append([cmd.x, cmd.y])  
            last = [cmd.x, cmd.y]
            
    newPline = Polyline(pts)
    return newPline

In [623]:
page = document(288, 100, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40))
factor = 72 / f.density

letter = scribble(f, "h").scale(factor).translate(100,100)

lines = pen.polyline(letter)

page.place(lines)

show(page)

In [634]:
page = document(700, 300, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40))
factor = 144 / f.density
word = ["s", "c", "r", "i", "b", "b", "l", "e"]

for i in range(len(word)):
    letter = scribble(f, word[i]).scale(factor).translate(50+(i*100),200)
    lines = pen.polyline(letter)
    page.place(lines)
    
show(page)

For the final composition, I converted all the lines in each character into scribbles using polylines with normalized point values. I really like how this version looks! The version below is just to show what it could look like with less scribbling. With some more time, I'd like to look into adding less points and joining them up in a smooth path, which i think would look nice! But may be a bit similar to just modifying the x and y values of the original points using normal(). I wasn't able to make a functional combine_path function since this is working with polylines, so the letters are evenly spaced, which is not the best. But I still think it turned out well, very close to what I envisioned!

In [636]:
def scribble(f, c):
    glyph_path = glyphcommands(f, c)
    pts = []
    start = []
    last = []
    for cmd in glyph_path:
        if type(cmd) == moveto:
            start = [cmd.x, cmd.y]
            last = [cmd.x, cmd.y]
            pts.append(start)
            
        elif type(cmd) == closepath:
            pts.append(start)
            
        elif hasattr(cmd, "x"):
            for i in range(5):
                pts.append([last[0] + ((cmd.x-last[0])/10)*i + normal(0,10), last[1] + ((cmd.y-last[1])/10)*i + normal(0,10)])
            pts.append([cmd.x, cmd.y])  
            last = [cmd.x, cmd.y]
            
    newPline = Polyline(pts)
    return newPline

In [637]:
page = document(700, 300, 'pt').addpage()
pen = shape().stroke(rgb(40, 40, 40))
factor = 144 / f.density
word = ["s", "c", "r", "i", "b", "b", "l", "e"]

for i in range(len(word)):
    letter = scribble(f, word[i]).scale(factor).translate(50+(i*100),200)
    lines = pen.polyline(letter)
    page.place(lines)
    
show(page)